In [1]:
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [2]:
# !pip install langchain-community langchain-core

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)

In [4]:
from langchain.chains import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate
)
from langchain.memory import ConversationBufferMemory

class Agent:
    def __init__(self, name, system_prompt, llm):
        self.name = name
        self.llm = llm
        self.system_prompt = system_prompt
        # Create a chat prompt that includes 'conversation_history'
        self.chat_prompt = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(self.system_prompt),
            # Include the conversation history and the last message in the prompt
            HumanMessagePromptTemplate.from_template("{conversation_history}\n{input}")
        ])
        self.chain = LLMChain(
            llm=self.llm,
            prompt=self.chat_prompt,
            verbose=False
        )

    def respond(self, input_text, conversation_history):
        response = self.chain.predict(
            input=input_text,
            conversation_history=conversation_history
        )
        return response

In [5]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.7,
    openai_api_key=os.getenv("OPENAI_API_KEY"))

/var/folders/hm/byc5swds1j18k5jny9rr6xp00000gn/T/ipykernel_33457/1446886985.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(


In [ ]:
customer_proxy_agent = Agent(
    name="Customer_Proxy",
    system_prompt="""Acts as a representative of the customer. Communicates the customer's high-level requirements, vision, and goals to the Product Owner. Provides an overarching view of the product’s purpose, expected functionalities, and outcomes. 
    Coordinates with the Product Owner to ensure the product vision aligns with customer needs.""",
    llm=llm
)

/var/folders/hm/byc5swds1j18k5jny9rr6xp00000gn/T/ipykernel_33457/3812591458.py:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.chain = LLMChain(


In [ ]:
product_owner_agent = Agent(
    name="Product_Owner",
    system_prompt="""As the Product Owner, you definethe product's vision, goals, and high-level requirements. After gathering the customer's problem statement:
    - Translate requirements into high-level product goals, key performance indicators (KPIs), and essential user stories.
    - Then work with Customer Proxy to refine the product vision and ensure alignment with customer needs.
    - Once done, work with the Scrum Master to establish a roadmap, prioritizing features and creating the initial product backlog.
    - Once the backlog and initial sprint plan is received from the Scrum Master,
        - Review the product backlog and initial sprint plan based on customer needs and business value.
        - Prepare a report on the product backlog and sprint plan for the customer and stakeholders. The Report will include the product vision, goals, high-level requirements, Number of Sprints, Number of Story Points for each sprint, estimated duration of each sprint, total project plan.
    - Oversee the initial product vision and ensure alignment with customer needs as the development plan is refined.""",
    llm=llm
)

In [ ]:
scrum_master_agent = Agent(
    name="Scrum_Master",
    system_prompt="""As the Scrum Master, you facilitate the Scrum process, enabling seamless collaboration. During initial planning:
    - Review the product vision, high-level product goals, key performance indicators (KPIs), and essential user stories from the Product Owner.
    - Additionally, Outline key tasks and structure the initial project timeline.
    - Communicate everything to the Project Manager, guiding initial sprint planning.
    - Once the specific project plan is received from the project manager and use that to create the initial product backlog and sprint plan. Below are the tasks to be performed:
       - Create a Final Product baclog and divide the backlog into sprints.
       - Each sprint should have a backlog, a goal, estiamtion of story points, and duration.
       - Prepare a report on the product backlog and sprint plan to send it to Product owner for review. The Report will include the Number of Sprints, Number of Story Points for each sprint, Backlog and goals for each sprint, estimated duration of each sprint and total project plan.
    """,
    llm=llm
)

In [ ]:
project_manager_agent = Agent(
    name="Project_Manager",
    system_prompt="""As the Project Manager, your primary responsibility is to create and coordinate the overall project plan. During the initial planning phase:
    - Receives and reviews the product vision, high-level product goals, key performance indicators (KPIs), and essential user stories, key tasks, and initial project timeline from the Scrum Master.
    - Coordinate with each engineer (Requirements, System, Software, Test, and Documentation) to gather user stories, effort estimates, and specific sprint plan.
    - Once the specific sprint plan is received from each engineer,
      - Complie the response from all engineers.
      - Make sure the scrum master can calcuate the backlog, goal, estimation of story points for each sprint and duration.
    """,
    llm=llm
)

In [ ]:
requirement_engineer_agent = Agent(
    name="Requirement_Engineer",
    system_prompt=("""
    As the Requirement Engineer, you are responsible for defining detailed user stories and requirements. During the planning phase:
    - Convert intial sprint plan, high-level customer requirements, key performance indicators (KPIs) into structured user stories based on their requirements, with unique identifiers and defined acceptance criteria for each user story.
    - Use past productivity rates to estimate theeffortand effort for each user story, factoring in the complexity of requirements.
    - Divide the user stories into sprints based on the project scope and priorities.
    - Calculate total effort for each sprint based on the volume of user stories and average productivity, ensuring alignment with overall project scope.
    - Report the detailed user stories and estimatedeffortto the Project Manager, providing a high-level summary and key requirements for the planning phase.
    - Calculate the {effort} to create the requirements document in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of requirements for the use-cases needed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many requirements can be completed every day} 
                            - step 3. effort = {work}/{productivity}
    """
    ),
    llm=llm
)

In [ ]:
system_engineer_agent = Agent(
    name="System_Engineer",
    system_prompt=(
        """As the System Engineer, you design the system architecture and data model based on requirements provided. During the planning phase:
    - Analyze user stories to create a detailed system design document and Data Model Document.
    - Document each part of the design with unique identifiers, estimatingeffortand effort using historical productivity data.
    - Calculate total effort based on the estimated pages required for design documentation.
    - Provide the Project Manager with a summary of the system design, including estimatedeffortfor completion, a list of key components, and any identified complexities.
    - Calculate the {effort} to create the design document in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of pages in the design document for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many pages can be completed every day} 
                            - step 3. effort = {work}/{productivity}
    """
    ),
    llm=llm
)

In [ ]:
software_engineer_agent = Agent(
    name="Software_Engineer",
    system_prompt=(
       """As the Software Engineer, you implement features and functionality based on the system design. During the planning phase:
    - Break down the system design into coding tasks, each with an estimated number of lines of code (SLOC).
    - Use past productivity data to calculate effort requirements for coding tasks, code reviews, and testing.
    - Provide the Project Manager with a summary of coding tasks, total estimated SLOC, and effort requirements for each major feature.
    - Ensure estimates include effort for Unit Testing, refactoring, and review meetings as needed.
    - calculate the {effort} to write the source code (SLOC) in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of lines of code (SLOC) that will be created for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many SLOC can be completed every day} 
                            - step 3. effort = {work}/{productivity}
    """
    ),
    llm=llm
)

In [ ]:
test_engineer_agent = Agent(
    name="Test_Engineer",
    system_prompt=(
        """As the Test Engineer, you are responsible for verifying the quality and functionality of the product. During the planning phase:
    - Develop a test plan with user stories defining individual test cases, including acceptance criteria.
    - Estimate effort for test case creation, execution, defect tracking, and retesting based on past productivity data.
    - Report to the Project Manager with a structured test plan, estimated number of test cases, and overall effort required for the testing phase.
    - Summarize key testing tasks, including preparation, defect resolution, and documentation of results.
    calculate the {effort} to write and execute the test cases in the test plan in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of test cases that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many test cases can be completed every day} 
                            - step 3. effort = {work}/{productivity} 
    """
    ),
    llm=llm
)

In [ ]:
documentation_engineer_agent = Agent(
    name="Documentation_Engineer",
    system_prompt=(
        """As the Documentation Engineer, you create comprehensive user and training documentation. During the planning phase:
    - Outline documentation tasks, including user guides and training materials, based on user stories and completed product features.
    - Estimate the effort required for each documentation task based on historical productivity data, including effort for review and rework.
    - Provide the Project Manager with a summary report of documentation tasks, estimated total pages, and effort requirements.
    - Highlight any dependencies on development or testing and plan for incremental updates as features are completed.
    calculate the {effort} to write documentation pages in the documentation adn traingin phase in hours, days, weeks, or months, follow the steps below, having cosideration a day has 8 hours:
                            - step 1. work = {the estimate of total number of documentation pages that will be created adn executed for the sofwtare project} 
                            - step 2. productivity rate = {estimate the complexities of the work and estimate how many pages can be completed every day} 
                            - step 3. effort = {work}/{productivity} 
    """
    ),
    llm=llm
)

In [15]:
conversation_prompts = {
    "Project_Manager":(),
    "Requirement_Engineer": (
        "The Scrum Master has provided the initial sprint planning requirements based on the Product Owner’s input. "
    "As the Requirements Engineer, please create detailed user stories for requirements gathering and documentation, witheffortestimates. "
    "Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the requirement engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the requirement engineering phase."

    "The provided historical data for the requirement engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the requirement engineering phase:"
    "Here is the amount of work and productivity rate for the requirement engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           112 Req                 5 Req/Hour"
        "Review Requirments                                                            "
        "       Preparation for Review                112  Req               18 Req/Hour"
        "       Review Meeting                        112 Req                28 Req/Hour"
        "Rework                                       136 defects            10 defects/Hour"

        "Project 2"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           389 Req                 3 Req/Hour"
        "Review Requirments                                                      "
        "       Preparation for Review                389  Req               5 Req/Hour"
        "       Review Meeting                        389 Req                8 Req/Hour"
        "Rework                                       413 defects            5 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Requirement                           210 Req                 4 Req/Hour"
        "Review Requirments                                                      "
        "       Preparation for Review                210  Req               12 Req/Hour"
        "       Review Meeting                        210 Req                18 Req/Hour"
        "Rework                                       239 defects            8 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "System_Engineer": (
        "The Requirements Engineer has completed the initial user stories. Based on this, please draft user stories for system design and "
    "data model creation, providingeffortestimates for each task. Reference historical data for accuracy and outline a few sample user stories "
    "Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the system engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the requirement engineering phase."

    "The provided historical data for the system engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the system engineering phase:"    
    "Here is the amount of work and productivity rate for the system engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Analysis Document                      66 Pages                 5 Pages/Hour"
        "Review AD                                                            "
        "       Preparation for AD                 66 Pages                    4 Pages/Hour"
        "       Review Meeting                        66 Pages                9 Pages/Hour"
        "Rework                                       88 defects            6 defects/Hour"
        "Write Design Document                       123 defects                 6 Page/Hour"
        "Review DD                                                            "
        "       Preparation for DD                 89 Pages              4 Req/Hour"
        "       Review Meeting                        89 Pages                8 Pages/Hour"
        "Rework                                       188 defects            5 defects/Hour"
        "Write Data Model (DM)                        28 Pages               1page/4hours"
        "Review DM                                                                     "
        "       Preparation for DM                 28 Pages                   4 page/hour"
        "       Review Meeting                        28 Pages                5 page/hour"
        "Rework                                       88 defects            6 defects/Hour"

        "Project 2"
        "Task                                                          Amount of Work      Productivity Rate"
        "Write Analysis/Design Document                                     234 Pages                 4 Pages/Hour"
        "Review Analysis/Design Document                                                           "
        "       Preparation for Analysis/Design Document                   234 Pages              5 Pages/Hour"
        "       Review Meeting                                             234 Pages               8 Pages/Hour"
        "Rework                                                             509 defects            7 defects/Hour"
        "Write Data Model (DM)                                             62 Pages               1 page/ hour"
        "Review DM                                                                     "
        "       Preparation for DM                    62 Pages                 4 page/hour"
        "       Review Meeting                        62 Pages                 10 page/hour"
        "Rework                                       378 defects            6 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Analysis Document                      172 Pages                 5 Pages/Hour"
        "Review AD                                                            "
        "       Preparation for AD                   172 Pages                    5 Pages/Hour"
        "       Review Meeting                        172 Pages                9 Pages/Hour"
        "Rework                                       347 defects            7 defects/Hour"
        "Write Design Document                       220 Pages                 5 Pages/Hour"
        "Review DD                                                            "
        "       Preparation for DD                    220 Pages              5 Pages/Hour"
        "       Review Meeting                        220 Pages               8 Pages/Hour"
        "Rework                                       472 defects            6 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Software_Engineer": (
        "The System Engineer has finalized the design phase user stories. Please create user stories for coding tasks, estimating theeffortrequired for each. "
    "Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the software engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the requirement engineering phase."

    "The provided historical data for the software engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the software engineering phase:"
    "Here is the amount of work and productivity rate for the software engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Code                                  3800 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            256 test cases           10 Test Cases/Day"
        "       Fix Found Defects                     179 Defects              8 Defects/Day"
        "       Test Fixed Defects                    179 Defects              12 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       3800 SLOC                 113 SLOC/Hour"
        "       Code Inspection Meeting               3800 SLOC                 189 SLOC/Hour"
        "Rework                                       188 defects                5 defects/Hour"

        "Project 2"
        "Task                                    Amount of Work           Productivity Rate"
        "Write Code                                  9725 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            1045 test cases           5 Test Cases/Day"
        "       Fix Found Defects                     902 Defects              12 Defects/Day"
        "       Test Fixed Defects                    902 Defects              8 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       9725 SLOC                 90 SLOC/Hour"
        "       Code Inspection Meeting               9725 SLOC                 135 SLOC/Hour"
        "Rework                                       1230 defects                5 defects/Hour"

        "Project 3"
        "Task                                    Amount of Work           Productivity Rate"
        "Write Code                                  3800 SLOC                 5 SLOC/Hour"
        "Unit Testing                                                                     "
        "       Prepare/Execute Test Cases            332 test cases           25 Test Cases/Day"
        "       Fix Found Defects                     266 Defects              10 Defects/Day"
        "       Test Fixed Defects                    266 Defects              10 Defects/Day"
        "Code Inspection                                                                   "
        "       Preparation for Code Inspection       3800 SLOC                 102 SLOC/Hour"
        "       Code Inspection Meeting               3800 SLOC                 102 SLOC/Hour"
        "Rework                                       334 defects                5 defects/Hour"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Test_Engineer": (
        "The Software Engineer has outlined user stories for coding tasks. Based on this, please prepare user stories for test planning and execution. "
    "Estimateeffortfor writing test cases, execution, and rework based on historical test data. Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the software engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the test engineering phase."

    "The provided historical data for the test engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the test engineering phase:"
    "Here is the amount of work and productivity rate for the test engineering phase for the previous projects:"

        "Project 1"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    166 pages            5 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            166 Pages           4 Pages/Hour"
        "       Review Meeting                    166 Pages            8 Pages/Hour"
        "Rework                                   255 defects          5 defects/Hour"
        "Execute Test Plan                        215 Test Cases     9Test Cases/day"
        "Fix Found Defects                        213 Defects          14 Defects/day"
        "Test Fixed Defects                       213 Defects          17 Defects/day"

        "Project 2"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    417 pages            8 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            417 Pages           5 Pages/Hour"
        "       Review Meeting                    417 Pages            10 Pages/Hour"
        "Rework                                   502 defects          5 defects/Hour"
        "Execute Test Plan                        1167 Test Cases      8 Test Cases/day"
        "Fix Found Defects                        645 Defects          5 Defects/day"

        "Project 3"
        "Task                                    Amount of Work      Productivity Rate"
        "Write Test Plan (TP)                    205 pages            8 Pages/Day"
        "Review TP                                                            "
        "       Preparation for Review            205 Pages           5 Pages/Hour"
        "       Review Meeting                    205 Pages            10 Pages/Hour"
        "Rework                                   101 defects          4 defects/Hour"
        "Execute Test Plan                        194 Test Cases      8 Test Cases/day"
        "Fix Found Defects                        113 Defects          5 Defects/day"
        "Test Fixed Defect                        113 Defects          10 Defects/day"
    
    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Documentation_Engineer": (
        "The Test Engineer has completed user stories for testing. Please create user stories for documentation tasks, "
    "Provide a structured report with user stories (only give the summary of a list of user stories but give complete list of user stories to project manager),effortestimates, and effort calculations for the software engineering phase to support sprint planning."
    "At the end summaries the total number of user stories, total number of requirements and the total effort for the documentation engineering phase."

    "The provided historical data for the documentation engineering phase is for the waterfall model, and the productivity rate is based on the number of requirements per hour. But you need to relavant this data to the agile model. "
    "You may use the following information to estimate the Amount of work, Productivity Rate and efforts for the documentation engineering phase:"
    "Here is the amount of work and productivity rate for the documentation engineering phase for the previous projects:"

    "Project 1"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                     134 Pages                 4 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            134 Pages               5 Pages/Hour"
    "       Review Meeting                    134 Pages                7 Pages/Hour"
    "Rework                                   174 defects            6 defects/Hour"

    "Project 2"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      389 Pages                 6 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            389 Pages                5 Pages/Hour"
    "       Review Meeting                    389 Pages                7 Pages/Hour"
    "Rework                                   532 defects            4 defects/Hour"

    "Project 3"
    "Task                                    Amount of Work      Productivity Rate"
    "User Documentation                      177 Pages                 5 Pages/Hour"
    "Review User Documentation                                                    "
    "       Preparation for Review            177 Pages                5 Pages/Hour"
    "       Review Meeting                    177 Pages                7 Pages/Hour"
    "Rework                                   236 defects            5 defects/Hour"

    "Once completed, report back to me so I can proceed with the next steps in the project plan."
    ),
    "Scrum_Master": (
       "The outputs from all engineers have been received. I have compiled a complete backlog with user stories, time, and work estimates "
    "for each phase. This backlog is now ready for review and further structuring for the initial Scrum planning. "
    "Please find the backlog summary with user stories, effort estimates, and phase-specific effort estimates in the report attached. PLea review it and generate the final backlog and draft an approximate number of sprints required to complete the project and the user stories to be accomplished in each sprint."
    "Once done, report back the summary of the total number of sprints, total number of user stories and the total effort for the project."
    ),
    "Product_Owner": (
        "The Project Manager has compiled a complete backlog, which I have reviewed and structured for the initial Scrum plan. "
    "Based on the total user stories and effort estimates, I have drafted an approximate number of sprints required to complete the project. "
    "The backlog items have been divided into sprints, with rough sprint goals set for each. Please review this structured plan as we prepare for sprint kickoff."
    "Once you have finalised and prioritised the user stories for each sprint, report back the total number of sprints, total number of user stories and the total effort for the project."
    ),
    "Customer_Proxy":("The Scrum Master has provided an initial Scrum plan. I have reviewed and prioritized the user stories in the backlog, adjusting the sprint "
    "plan as necessary to align with our product goals. The final sprint breakdown, including prioritized user stories and sprint objectives, is now ready. "
    "This report includes the sprint goals, total number of sprints, and estimated completion timeline for review by the Customer Proxy.")
}

In [16]:
# Initialize the shared conversation history
conversation_history = ""

# Initial message from Customer Proxy
initial_message = (
    "I want to create an AI-powered computer vision system that detects anomalies in the skin, such as cancer and psoriasis. "
    "The system should process high-resolution images, provide real-time diagnostics, and support doctors with detailed reports. "
    "This project should prioritize accuracy, speed, and usability for healthcare providers."
    "Please document the initial product vision, objectives, and KPIs, including primary user stories that will guide development and outline a high-level product backlog focused on prioritized features and metrics to assess product value. "
)

# Add the initial message to the conversation history
conversation_history += f"Customer_Proxy: {initial_message}\n"
message_count = 1


# Customer Proxy sends message to Product Owner
product_owner_response = product_owner_agent.respond(initial_message, conversation_history=conversation_history)
print(f"{product_owner_agent.name}:\n{product_owner_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Customer Proxy and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Customer Proxy responds
    customer_proxy_response = customer_proxy_agent.respond(product_owner_response, conversation_history=conversation_history)
    print(f"{customer_proxy_agent.name}:\n{customer_proxy_response}\n")
    user_input = input("Press 'Enter' to continue the conversation between Customer Proxy and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Project Manager responds
    product_owner_response = product_owner_agent.respond(customer_proxy_response, conversation_history=conversation_history)
    print(f"{product_owner_agent.name}:\n{product_owner_response}\n")



second_message = (
    "I have gathered the requirements from the Customer Proxy."
    "structured an initial plan for sprint planning, including essential roles, estimated timeframes, and key milestones. Note that donot estimate the number of sprints and backlog items for each sprint unles your get all the project plan back from project manager. "
    "Please coordinate the initial sprint planning with the development team, ensuring each phase is planned around the structured backlog."
)

conversation_history += f"Product_Owner: {second_message}\n"


# Product Owner sends message to Scrum Master
scrum_master_response= scrum_master_agent.respond(second_message, conversation_history=conversation_history)
print(f"{scrum_master_agent.name}:\n{scrum_master_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    #  Product Owner responds
    product_owner_response = product_owner_agent.respond(scrum_master_response, conversation_history=conversation_history)
    print(f"{product_owner_agent.name}:\n{product_owner_response}\n")

    
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Product Owner, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break

    # Scrum Master responds
    scrum_master_response = scrum_master_agent.respond(product_owner_response, conversation_history=conversation_history)
    print(f"{scrum_master_agent.name}:\n{scrum_master_response}\n")
    

third_message = (
    "The Product Owner has shared the project vision and initial backlog for the skin anomaly detection system. "
    "I have structured an initial plan for sprint planning, including essential roles, estimated timeframes, and key milestones. "
    "As the Project Manager, please create a complete project plan with detailed user stories and effort requirements for each. Please only do this once you have all the project plan back from the development team. "
    "We aim to have a structured backlog based on user stories and ensure each story has clear effort estimates. "
    "Please coordinate with each engineer to gather use cases and relevant effort estimates."
)

conversation_history += f"Scrum_Master: {third_message}\n"
last_message = third_message

# Scrum Master sends message to Project Manager 
project_manager_response = project_manager_agent.respond(third_message, conversation_history=conversation_history)
print(f"{project_manager_agent.name}:\n{project_manager_response}\n")
while True:
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Project Manager, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Scrum Master responds
    scrum_master_response = scrum_master_agent.respond(project_manager_response, conversation_history=conversation_history)
    print(f"{scrum_master_agent.name}:\n{scrum_master_response}\n")
    user_input = input("Press 'Enter' to continue the conversation between Scrum Master and Project Manager, or type 'exit' to end it: ")
    if user_input.lower() == 'exit':
        break
    # Project Manager responds
    project_manager_response = project_manager_agent.respond(scrum_master_response, conversation_history=conversation_history)
    print(f"{project_manager_agent.name}:\n{project_manager_response}\n")






KeyboardInterrupt: 

In [ ]:
agents = [
    project_manager_agent,
    requirement_engineer_agent,
    system_engineer_agent,
    software_engineer_agent,
    test_engineer_agent,
    documentation_engineer_agent,
    scrum_master_agent,
    product_owner_agent,
    customer_proxy_agent
]
# Set the maximum number of messages and the message count
max_messages = 20
exit_loop = False  # Flag to control breaking out of both loops

# Start the conversation loop
while message_count < max_messages and not exit_loop:
    for agent in agents:
        user_input = input("Press 'Enter' to continue the conversation between the group, or type 'exit' to end it: ")
        if user_input.lower() == 'exit':
            exit_loop = True
            break  # Breaks out of the inner loop

        # Project Manager sends the specific prompt to the agent
        prompt = conversation_prompts[agent.name]
        # Project Manager sends the prompt as a message
        pm_message = prompt
        conversation_history += f"{project_manager_agent.name}: {pm_message}\n"

        # Agent responds to the last message with access to the conversation history
        agent_response = agent.respond(
            input_text=last_message,
            conversation_history=conversation_history
        )

        # Print the agent's response
        print(f"{agent.name}:\n{agent_response}\n")

        # Update the conversation history
        conversation_history += f"{agent.name}: {agent_response}\n"
        # Update the last message
        last_message = agent_response
        message_count += 1

        if message_count >= max_messages or exit_loop:
            break  # Breaks out of the inner loop if conditions are met

# Exit the program after breaking out of both loops
print("Conversation ended.")